In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Import dependencies

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import cobra

from sklearn.decomposition import PCA

from src.gem.yeast8model import Yeast8Model

Initialise model

In [ ]:
glc_exch_rate = 16.89

wt = Yeast8Model("../data/gemfiles/ecYeastGEM_batch_8-6-0.xml")
# Default: lots of glucose
wt.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)

Generate & dump or load data

In [ ]:
exch_rate_dict = {
    "r_1714": np.linspace(0, 2*8.6869, 3), # glucose
    "r_1654": np.linspace(0, 2*1.4848, 3), # ammonium
}

In [ ]:
ablation_result_array = wt.usgfluxes_grid(exch_rate_dict)

# Dump data
with open('../data/interim/ec_usg_glc_amm.pkl', 'wb') as handle:
    pickle.dump(ablation_result_array, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Load if saved
with open('../data/interim/ec_usg_glc_amm.pkl', 'rb') as handle:
    ablation_result_array = pickle.load(handle)

Adjust data variable dimensions

In [ ]:
ablation_result_1d = ablation_result_array.ravel()
multicond_enz_use_array = np.concatenate(ablation_result_1d)
multicond_enz_use_array.shape

PCA

In [ ]:
pca = PCA()
Xt = pca.fit_transform(multicond_enz_use_array)
pca1 = Xt[:, 0]
pca2 = Xt[:, 1]

In [ ]:
print(np.cumsum(pca.explained_variance_ratio_))

In [ ]:
# Color dots by biomass components, using the default cycle.
# Original = C0, lipid = C1, etc.
num_components = 8
color_dict = dict(zip(
    list(range(num_components)),
    ['C' + str(num) for num in range(num_components)]
))
color_list = [color_dict[el] for el in (np.arange(len(pca1)) % num_components)]

fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(pca1, pca2, color=color_list)
ax.set_xlabel("PC1")
ax.set_ylabel("PC2")